In [19]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Dict, Union

import logging
import pandas as pd
from pathlib import Path

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#import xarray

from tqdm import tqdm

LOGGER = logging.getLogger(__name__)

#### Load rainfall data prepared for MTS-LSTM paper

In [13]:
#parameters
key_list = ['in_highres', 'out_highres', 'in_lowres', 'out_lowres']

HR_SEQLEN = 336
LR_SEQLEN = 365

BATCH_SIZE = 256



In [4]:
def load_rainfall_data(in_HR_fname, in_LR_fname, out_HR_fname, out_LR_fname):
    x_d_1H_array = torch.tensor(np.load(in_HR_fname))
    x_d_1D_array = torch.tensor(np.load(in_LR_fname))
    y_1H_array = torch.tensor(np.load(out_HR_fname))
    y_1D_array = torch.tensor(np.load(out_LR_fname))
    
    print("in_HR: ", x_d_1H_array.shape, x_d_1H_array.dtype)
    print("in_LR: ",x_d_1D_array.shape, x_d_1D_array.dtype)
    print("out_HR: ", y_1H_array.shape, y_1H_array.dtype)
    print("out_LR: ",y_1D_array.shape, y_1D_array.dtype)
    
    data_dict = dict(zip(key_list,[x_d_1H_array, y_1H_array, x_d_1D_array,  y_1D_array]))
    
    return data_dict
    
    
    
def load_lookuptable(HR_lookup_fname, LR_lookup_fname):
    """Metadata specifically for rainfall data"""
    H_lookup = np.load(HR_lookup_fname)
    D_lookup = np.load(LR_lookup_fname)
    
    return H_lookup, D_lookup



    
    
    

In [5]:
#training data

train_datadict = load_rainfall_data("x_d_1H_array.npy", "x_d_1D_array.npy", "y_1H_array.npy", "y_1D_array.npy")
H_lookup_train, D_lookup_train = load_lookuptable("H_lookup.npy", "D_lookup.npy")

in_HR:  torch.Size([87648, 16]) torch.float32
in_LR:  torch.Size([3652, 5]) torch.float32
out_HR:  torch.Size([87648, 1]) torch.float32
out_LR:  torch.Size([3652, 1]) torch.float32


In [6]:
#validation data

validation_datadict = load_rainfall_data("x_d_1H_array_val.npy", "x_d_1D_array_val.npy", "y_1H_array_val.npy", "y_1D_array_val.npy")
H_lookup_val, D_lookup_val = load_lookuptable("H_lookup_val.npy", "D_lookup_val.npy")

in_HR:  torch.Size([35016, 16]) torch.float32
in_LR:  torch.Size([1459, 5]) torch.float32
out_HR:  torch.Size([35016, 1]) torch.float32
out_LR:  torch.Size([1459, 1]) torch.float32


In [7]:
#test data

test_datadict = load_rainfall_data("x_d_1H_array_test.npy", "x_d_1D_array_test.npy", "y_1H_array_test.npy", "y_1D_array_test.npy")
H_lookup_test, D_lookup_test = load_lookuptable("H_lookup_test.npy", "D_lookup_test.npy")

in_HR:  torch.Size([70104, 16]) torch.float32
in_LR:  torch.Size([2921, 5]) torch.float32
out_HR:  torch.Size([70104, 1]) torch.float32
out_LR:  torch.Size([2921, 1]) torch.float32


### Custom dataset class

In [11]:
class TemporalDS_rainfall(Dataset):
    def __init__(self, data_dict, hr_seq_len, lr_seq_len, hlkup, dlkup):
        self.datadict = data_dict
        self.hr_seq_len = hr_seq_len
        self.lr_seq_len = lr_seq_len
        self.freq_factor = 24.0
        self.hlookup = hlkup
        self.dlookup = dlkup
        self.num_samples = self.hlookup.shape[0] #hardcoded right now for this example DS
        
        
    def __getitem__(self, index)-> Dict[str, torch.Tensor]:
        hr_idx = self.hlookup[index]
        lr_idx = self.dlookup[index]
        
        
        sample = {}
        
        #populate for high res first (i.e. hourly)
        sample[f'x_d_1H'] = self.datadict['in_highres'][hr_idx - self.hr_seq_len + 1: hr_idx+1]
        sample[f'y_1H'] = self.datadict['out_highres'][hr_idx - self.hr_seq_len + 1: hr_idx+1]
        
        sample[f'x_d_1D'] = self.datadict['in_lowres'][lr_idx - self.lr_seq_len + 1: lr_idx+1]
        sample[f'y_1D'] = self.datadict['out_lowres'][lr_idx - self.lr_seq_len + 1: lr_idx+1]

        return sample

    def __len__(self):
        return self.num_samples

In [12]:
ds_train = TemporalDS_rainfall(train_datadict, hr_seq_len=HR_SEQLEN, lr_seq_len=LR_SEQLEN, hlkup=H_lookup_train, dlkup=D_lookup_train)
ds_val = TemporalDS_rainfall(validation_datadict, hr_seq_len=HR_SEQLEN, lr_seq_len=LR_SEQLEN, hlkup=H_lookup_val, dlkup=D_lookup_val)
ds_test = TemporalDS_rainfall(test_datadict, hr_seq_len=HR_SEQLEN, lr_seq_len=LR_SEQLEN, hlkup=H_lookup_test, dlkup=D_lookup_test)


In [14]:
dl_train = DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True)
dl_val = DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=False)
dl_test = DataLoader(ds_test, batch_size=1, shuffle=False)

#### lstm class and util functions

In [28]:
from neuralhydrology.utils.config import Config
from neuralhydrology.utils.samplingutils import sample_pointpredictions
from neuralhydrology.datautils.utils import get_frequency_factor, sort_frequencies
from neuralhydrology.modelzoo.head import get_head

import neuralhydrology.training.loss as loss

In [29]:
class BaseModel(nn.Module):
    
    # specify submodules of the model that can later be used for finetuning. Names must match class attributes
    module_parts = []

    def __init__(self, cfg: Config):
        super(BaseModel, self).__init__()
        self.cfg = cfg
        self.output_size = len(cfg.output_dim)
        # if cfg.head.lower() == 'gmm':
        #     self.output_size *= 3 * cfg.n_distributions
        # elif cfg.head.lower() == 'cmal':
        #     self.output_size *= 4 * cfg.n_distributions
        # elif cfg.head.lower() == 'umal':
        #     self.output_size *= 2

    def sample(self, data: Dict[str, torch.Tensor], n_samples: int) -> Dict[str, torch.Tensor]:
        
        return sample_pointpredictions(self, data, n_samples)

    def forward(self, data: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        
        raise NotImplementedError

In [30]:
class MTSLSTM(BaseModel):
    
    # specify submodules of the model that can later be used for finetuning. Names must match class attributes
    module_parts = ['lstms', 'transfer_fcs', 'heads']

    def __init__(self, cfg: Config):
        super(MTSLSTM, self).__init__(cfg=cfg)
        self.lstms = None
        self.transfer_fcs = None
        self.heads = None
        self.dropout = None

        self._slice_timestep = {}
        self._frequency_factors = []

        self._seq_lengths = cfg.seq_length
        self._is_shared_mtslstm = self.cfg.shared_mtslstm  # default: a distinct LSTM per timescale
        self._transfer_mtslstm_states = self.cfg.transfer_mtslstm_states  # default: linear transfer layer
        transfer_modes = [None, "None", "identity", "linear"]
        if self._transfer_mtslstm_states["h"] not in transfer_modes \
                or self._transfer_mtslstm_states["c"] not in transfer_modes:
            raise ValueError(f"MTS-LSTM supports state transfer modes {transfer_modes}")

        if len(cfg.use_frequencies) < 2:
            raise ValueError("MTS-LSTM expects more than one input frequency")
        self._frequencies = sort_frequencies(cfg.use_frequencies)

        # start to count the number of inputs
        input_sizes = len(cfg.static_attributes + cfg.hydroatlas_attributes + cfg.evolving_attributes)

        # if is_shared_mtslstm, the LSTM gets an additional frequency flag as input.
        if self._is_shared_mtslstm:
            input_sizes += len(self._frequencies)

        if cfg.use_basin_id_encoding:
            input_sizes += cfg.number_of_basins
        if cfg.head.lower() == "umal":
            input_sizes += 1

        if isinstance(cfg.dynamic_inputs, list):
            input_sizes = {freq: input_sizes + len(cfg.dynamic_inputs) for freq in self._frequencies}
        else:
            if self._is_shared_mtslstm:
                raise ValueError(f'Different inputs not allowed if shared_mtslstm is used.')
            input_sizes = {freq: input_sizes + len(cfg.dynamic_inputs[freq]) for freq in self._frequencies}

        if not isinstance(cfg.hidden_size, dict):
            LOGGER.info("No specific hidden size for frequencies are specified. Same hidden size is used for all.")
            self._hidden_size = {freq: cfg.hidden_size for freq in self._frequencies}
        else:
            self._hidden_size = cfg.hidden_size

        if (self._is_shared_mtslstm
            or self._transfer_mtslstm_states["h"] == "identity"
            or self._transfer_mtslstm_states["c"] == "identity") \
                and any(size != self._hidden_size[self._frequencies[0]] for size in self._hidden_size.values()):
            raise ValueError("All hidden sizes must be equal if shared_mtslstm is used or state transfer=identity.")

        # create layer depending on selected frequencies
        self._init_modules(input_sizes)
        self._reset_parameters()

        # frequency factors are needed to determine the time step of information transfer
        self._init_frequency_factors_and_slice_timesteps()

    def _init_modules(self, input_sizes: Dict[str, int]):
        self.lstms = nn.ModuleDict()
        self.transfer_fcs = nn.ModuleDict()
        self.heads = nn.ModuleDict()
        self.dropout = nn.Dropout(p=self.cfg.output_dropout)
        for idx, freq in enumerate(self._frequencies):
            freq_input_size = input_sizes[freq]

            if self._is_shared_mtslstm and idx > 0:
                self.lstms[freq] = self.lstms[self._frequencies[idx - 1]]  # same LSTM for all frequencies.
                self.heads[freq] = self.heads[self._frequencies[idx - 1]]  # same head for all frequencies.
            else:
                self.lstms[freq] = nn.LSTM(input_size=freq_input_size, hidden_size=self._hidden_size[freq])
                self.heads[freq] = get_head(self.cfg, n_in=self._hidden_size[freq], n_out=self.output_size)

            if idx < len(self._frequencies) - 1:
                for state in ["c", "h"]:
                    if self._transfer_mtslstm_states[state] == "linear":
                        self.transfer_fcs[f"{state}_{freq}"] = nn.Linear(self._hidden_size[freq],
                                                                         self._hidden_size[self._frequencies[idx + 1]])
                    elif self._transfer_mtslstm_states[state] == "identity":
                        self.transfer_fcs[f"{state}_{freq}"] = nn.Identity()
                    else:
                        pass

    def _init_frequency_factors_and_slice_timesteps(self):
        for idx, freq in enumerate(self._frequencies):
            if idx < len(self._frequencies) - 1:
                frequency_factor = get_frequency_factor(freq, self._frequencies[idx + 1])
                if frequency_factor != int(frequency_factor):
                    raise ValueError('Adjacent frequencies must be multiples of each other.')
                self._frequency_factors.append(int(frequency_factor))
                # we want to pass the state of the day _before_ the next higher frequency starts,
                # because e.g. the mean of a day is stored at the same date at 00:00 in the morning.
                slice_timestep = int(self._seq_lengths[self._frequencies[idx + 1]] / self._frequency_factors[idx])
                self._slice_timestep[freq] = slice_timestep

    def _reset_parameters(self):
        if self.cfg.initial_forget_bias is not None:
            for freq in self._frequencies:
                hidden_size = self._hidden_size[freq]
                self.lstms[freq].bias_hh_l0.data[hidden_size:2 * hidden_size] = self.cfg.initial_forget_bias

    def _prepare_inputs(self, data: Dict[str, torch.Tensor], freq: str) -> torch.Tensor:
        """Concat all different inputs to the time series input"""
        suffix = f"_{freq}"
        # transpose to [seq_length, batch_size, n_features]
        x_d = data[f'x_d{suffix}'].transpose(0, 1)

        # concat all inputs
        if f'x_s{suffix}' in data and 'x_one_hot' in data:
            x_s = data[f'x_s{suffix}'].unsqueeze(0).repeat(x_d.shape[0], 1, 1)
            x_one_hot = data['x_one_hot'].unsqueeze(0).repeat(x_d.shape[0], 1, 1)
            x_d = torch.cat([x_d, x_s, x_one_hot], dim=-1)
        elif f'x_s{suffix}' in data:
            x_s = data[f'x_s{suffix}'].unsqueeze(0).repeat(x_d.shape[0], 1, 1)
            x_d = torch.cat([x_d, x_s], dim=-1)
        elif 'x_one_hot' in data:
            x_one_hot = data['x_one_hot'].unsqueeze(0).repeat(x_d.shape[0], 1, 1)
            x_d = torch.cat([x_d, x_one_hot], dim=-1)
        else:
            pass

        if self._is_shared_mtslstm:
            # add frequency one-hot encoding
            idx = self._frequencies.index(freq)
            one_hot_freq = torch.zeros(x_d.shape[0], x_d.shape[1], len(self._frequencies)).to(x_d)
            one_hot_freq[:, :, idx] = 1
            x_d = torch.cat([x_d, one_hot_freq], dim=2)

        return x_d

    def forward(self, data: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        """Perform a forward pass on the MTS-LSTM model.
        
        Parameters
        ----------
        data : Dict[str, torch.Tensor]
            Input data for the forward pass. See the documentation overview of all models for details on the dict keys.
        Returns
        -------
        Dict[str, torch.Tensor]
            Model predictions for each target timescale.
        """
        x_d = {freq: self._prepare_inputs(data, freq) for freq in self._frequencies}

        # initial states for lowest frequencies are set to zeros
        batch_size = x_d[self._frequencies[0]].shape[1]
        lowest_freq_hidden_size = self._hidden_size[self._frequencies[0]]
        h_0_transfer = x_d[self._frequencies[0]].new_zeros((1, batch_size, lowest_freq_hidden_size))
        c_0_transfer = torch.zeros_like(h_0_transfer)

        outputs = {}
        for idx, freq in enumerate(self._frequencies):
            if idx < len(self._frequencies) - 1:
                # get predictions and state up to the time step of information transfer
                slice_timestep = self._slice_timestep[freq]
                lstm_output_slice1, (h_n_slice1, c_n_slice1) = self.lstms[freq](x_d[freq][:-slice_timestep],
                                                                                (h_0_transfer, c_0_transfer))

                # project the states through a hidden layer to the dimensions of the next LSTM
                if self._transfer_mtslstm_states["h"] is not None:
                    h_0_transfer = self.transfer_fcs[f"h_{freq}"](h_n_slice1)
                if self._transfer_mtslstm_states["c"] is not None:
                    c_0_transfer = self.transfer_fcs[f"c_{freq}"](c_n_slice1)

                # get predictions of remaining part and concat results
                lstm_output_slice2, _ = self.lstms[freq](x_d[freq][-slice_timestep:], (h_n_slice1, c_n_slice1))
                lstm_output = torch.cat([lstm_output_slice1, lstm_output_slice2], dim=0)

            else:
                # for highest frequency, we can pass the entire sequence at once
                lstm_output, _ = self.lstms[freq](x_d[freq], (h_0_transfer, c_0_transfer))

            head_out = self.heads[freq](self.dropout(lstm_output.transpose(0, 1)))
            outputs.update({f'{key}_{freq}': value for key, value in head_out.items()})

        return outputs

In [31]:
def _get_predictions_and_loss(model, data, l_obj):
    predictions = model(data)
    loss = l_obj(predictions, data)
    return predictions, loss.item()

def _subset_targets(data, predictions, predict_last_n, freq):
    y_hat_sub = predictions[f'y_hat{freq}'][:, -predict_last_n:, :]
    y_sub = data[f'y{freq}'][:, -predict_last_n:, :]
    return y_hat_sub, y_sub


def _evaluate(model, loader, l_obj, frequencies):
    """Evaluate model"""
    predict_last_n = conf_obj.predict_last_n
    # if isinstance(predict_last_n, int):
    #     predict_last_n = {frequencies[0]: predict_last_n}  # if predict_last_n is int, there's only one frequency

    preds, obs = {}, {}
    losses = []
    with torch.no_grad():
        for data in loader:

            # for key in data:
            #     data[key] = data[key].to(self.device)
            predictions, loss = _get_predictions_and_loss(model, data, l_obj)

            for freq in frequencies:
                freq_key = f'_{freq}'
                y_hat_sub, y_sub = _subset_targets(data, predictions, predict_last_n[freq], freq_key)

                if freq not in preds:
                    preds[freq] = y_hat_sub.detach().cpu()
                    obs[freq] = y_sub.cpu()
                else:
                    preds[freq] = torch.cat((preds[freq], y_hat_sub.detach().cpu()), 0)
                    obs[freq] = torch.cat((obs[freq], y_sub.detach().cpu()), 0)

            losses.append(loss)

        for freq in preds.keys():
            preds[freq] = preds[freq].numpy()
            obs[freq] = obs[freq].numpy()

    # set to NaN explicitly if all losses are NaN to avoid RuntimeWarning
    mean_loss = np.nanmean(losses) if len(losses) > 0 and not all(np.isnan(l) for l in losses) else np.nan
    return preds, obs, mean_loss

In [35]:
class Regression(nn.Module):
    
    """Single-layer regression head with different output activations.
    
    Parameters
    ----------
    n_in : int
        Number of input neurons.
    n_out : int
        Number of output neurons.
    activation : str, optional
        Output activation function. Can be specified in the config using the `output_activation` argument. Supported
        are {'linear', 'relu', 'softplus'}. If not specified (or an unsupported activation function is specified), will
        default to 'linear' activation.
    """

    def __init__(self, n_in: int, n_out: int, activation: str = "linear"):
        super(Regression, self).__init__()

        # TODO: Add multi-layer support
        layers = [nn.Linear(n_in, n_out)]
        if activation != "linear":
            if activation.lower() == "relu":
                layers.append(nn.ReLU())
            elif activation.lower() == "softplus":
                layers.append(nn.Softplus())
            else:
                LOGGER.warning(f"## WARNING: Ignored output activation {activation} and used 'linear' instead.")
        self.net = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> Dict[str, torch.Tensor]:
        """Perform a forward pass on the Regression head.
        
        Parameters
        ----------
        x : torch.Tensor

        Returns
        -------
        Dict[str, torch.Tensor]
            Dictionary containing the model predictions in the 'y_hat' key.
        """
        return {'y_hat': self.net(x)}

In [69]:
LR_INPUT_DIM = 5
HR_INPUT_DIM = 16
INPUT_SIZE = {'1D': 5, '1H': 16}
OUTPUT_DIM = 1
HIDDEN_SIZE = {'1D': 20, '1H': 20}
INITIAL_FORGET_BIAS = 3
SHARED_MTSLSTM = False
OUTPUT_DROPOUT = 0.4

TRANSFER_MTSLSTM_STATES = {'h': 'linear', 'c': 'linear'}

USE_FREQUENCIES = ['1D', '1H']
OUTPUT_ACTIVATION = 'linear'

DEVICE = 'cpu'

SEQ_LENGTH = {'1D': 365, '1H': 336}
PREDICT_LAST_N = {'1D': 1, '1H': 24}

SLICE_TIMESTEP = {'1D': 14}
FREUENCY_FACTORS = [24]

#weights per target
TARGET_WEIGHTS = [1.0]





In [56]:
class AiBEDO_MTSLSTM(nn.Module):
    
    # specify submodules of the model that can later be used for finetuning. Names must match class attributes
    module_parts = ['lstms', 'transfer_fcs', 'heads']

    def __init__(self):
        super(AiBEDO_MTSLSTM, self).__init__()
        self.output_size = OUTPUT_DIM
        self.lstms = None
        self.transfer_fcs = None
        self.heads = None
        self.dropout = None

        self._slice_timestep = {}
        self._frequency_factors = []

        self._seq_lengths = SEQ_LENGTH
        self._is_shared_mtslstm = SHARED_MTSLSTM  # default: a distinct LSTM per timescale
        self._transfer_mtslstm_states = TRANSFER_MTSLSTM_STATES  # default: linear transfer layer
        transfer_modes = [None, "None", "identity", "linear"]
        if self._transfer_mtslstm_states["h"] not in transfer_modes \
                or self._transfer_mtslstm_states["c"] not in transfer_modes:
            raise ValueError(f"MTS-LSTM supports state transfer modes {transfer_modes}")

        # if len(cfg.use_frequencies) < 2:
        #     raise ValueError("MTS-LSTM expects more than one input frequency")
        self._frequencies = USE_FREQUENCIES

        # # start to count the number of inputs
        # input_sizes = len(cfg.static_attributes + cfg.hydroatlas_attributes + cfg.evolving_attributes)

        # # if is_shared_mtslstm, the LSTM gets an additional frequency flag as input.
        # if self._is_shared_mtslstm:
        #     input_sizes += len(self._frequencies)

        # if cfg.use_basin_id_encoding:
        #     input_sizes += cfg.number_of_basins
        # if cfg.head.lower() == "umal":
        #     input_sizes += 1

        # if isinstance(cfg.dynamic_inputs, list):
        #     input_sizes = {freq: input_sizes + len(cfg.dynamic_inputs) for freq in self._frequencies}
        # else:
        #     if self._is_shared_mtslstm:
        #         raise ValueError(f'Different inputs not allowed if shared_mtslstm is used.')
        #     input_sizes = {freq: input_sizes + len(cfg.dynamic_inputs[freq]) for freq in self._frequencies}
        
        
        self._input_sizes = INPUT_SIZE
        self._hidden_size = HIDDEN_SIZE

        if (self._is_shared_mtslstm
            or self._transfer_mtslstm_states["h"] == "identity"
            or self._transfer_mtslstm_states["c"] == "identity") \
                and any(size != self._hidden_size[self._frequencies[0]] for size in self._hidden_size.values()):
            raise ValueError("All hidden sizes must be equal if shared_mtslstm is used or state transfer=identity.")

        # create layer depending on selected frequencies
        self._init_modules(self._input_sizes)
        self._reset_parameters()

        # frequency factors are needed to determine the time step of information transfer
        #self._init_frequency_factors_and_slice_timesteps()
        self._frequency_factors = FREUENCY_FACTORS
        self._slice_timestep = SLICE_TIMESTEP

    def _init_modules(self, input_sizes: Dict[str, int]):
        self.lstms = nn.ModuleDict()
        self.transfer_fcs = nn.ModuleDict()
        self.heads = nn.ModuleDict()
        self.dropout = nn.Dropout(p=OUTPUT_DROPOUT)
        for idx, freq in enumerate(self._frequencies):
            freq_input_size = input_sizes[freq]

            if self._is_shared_mtslstm and idx > 0:
                self.lstms[freq] = self.lstms[self._frequencies[idx - 1]]  # same LSTM for all frequencies.
                self.heads[freq] = self.heads[self._frequencies[idx - 1]]  # same head for all frequencies.
            else:
                self.lstms[freq] = nn.LSTM(input_size=freq_input_size, hidden_size=self._hidden_size[freq])
                #self.heads[freq] = get_head(self.cfg, n_in=self._hidden_size[freq], n_out=self.output_size)
                self.heads[freq] = Regression(n_in=self._hidden_size[freq], n_out=self.output_size, activation=OUTPUT_ACTIVATION)
                
                

            if idx < len(self._frequencies) - 1:
                for state in ["c", "h"]:
                    if self._transfer_mtslstm_states[state] == "linear":
                        self.transfer_fcs[f"{state}_{freq}"] = nn.Linear(self._hidden_size[freq],
                                                                         self._hidden_size[self._frequencies[idx + 1]])
                    elif self._transfer_mtslstm_states[state] == "identity":
                        self.transfer_fcs[f"{state}_{freq}"] = nn.Identity()
                    else:
                        pass

    # def _init_frequency_factors_and_slice_timesteps(self):
    #     for idx, freq in enumerate(self._frequencies):
    #         if idx < len(self._frequencies) - 1:
    #             frequency_factor = get_frequency_factor(freq, self._frequencies[idx + 1])
    #             if frequency_factor != int(frequency_factor):
    #                 raise ValueError('Adjacent frequencies must be multiples of each other.')
    #             self._frequency_factors.append(int(frequency_factor))
    #             # we want to pass the state of the day _before_ the next higher frequency starts,
    #             # because e.g. the mean of a day is stored at the same date at 00:00 in the morning.
    #             slice_timestep = int(self._seq_lengths[self._frequencies[idx + 1]] / self._frequency_factors[idx])
    #             self._slice_timestep[freq] = slice_timestep
                
    

    def _reset_parameters(self):
        if INITIAL_FORGET_BIAS is not None:
            for freq in self._frequencies:
                hidden_size = self._hidden_size[freq]
                self.lstms[freq].bias_hh_l0.data[hidden_size:2 * hidden_size] = INITIAL_FORGET_BIAS
    
    def _prepare_inputs(self, data: Dict[str, torch.Tensor], freq: str) -> torch.Tensor:
        """Concat all different inputs to the time series input"""
        suffix = f"_{freq}"
        # transpose to [seq_length, batch_size, n_features]
        x_d = data[f'x_d{suffix}'].transpose(0, 1)

        # concat all inputs
        if f'x_s{suffix}' in data and 'x_one_hot' in data:
            x_s = data[f'x_s{suffix}'].unsqueeze(0).repeat(x_d.shape[0], 1, 1)
            x_one_hot = data['x_one_hot'].unsqueeze(0).repeat(x_d.shape[0], 1, 1)
            x_d = torch.cat([x_d, x_s, x_one_hot], dim=-1)
        elif f'x_s{suffix}' in data:
            x_s = data[f'x_s{suffix}'].unsqueeze(0).repeat(x_d.shape[0], 1, 1)
            x_d = torch.cat([x_d, x_s], dim=-1)
        elif 'x_one_hot' in data:
            x_one_hot = data['x_one_hot'].unsqueeze(0).repeat(x_d.shape[0], 1, 1)
            x_d = torch.cat([x_d, x_one_hot], dim=-1)
        else:
            pass

        if self._is_shared_mtslstm:
            # add frequency one-hot encoding
            idx = self._frequencies.index(freq)
            one_hot_freq = torch.zeros(x_d.shape[0], x_d.shape[1], len(self._frequencies)).to(x_d)
            one_hot_freq[:, :, idx] = 1
            x_d = torch.cat([x_d, one_hot_freq], dim=2)

        return x_d

    def forward(self, data: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        """Perform a forward pass on the MTS-LSTM model.
        
        Parameters
        ----------
        data : Dict[str, torch.Tensor]
            Input data for the forward pass. See the documentation overview of all models for details on the dict keys.
        Returns
        -------
        Dict[str, torch.Tensor]
            Model predictions for each target timescale.
        """
        x_d = {freq: self._prepare_inputs(data, freq) for freq in self._frequencies}

        # initial states for lowest frequencies are set to zeros
        batch_size = x_d[self._frequencies[0]].shape[1]
        lowest_freq_hidden_size = self._hidden_size[self._frequencies[0]]
        h_0_transfer = x_d[self._frequencies[0]].new_zeros((1, batch_size, lowest_freq_hidden_size))
        c_0_transfer = torch.zeros_like(h_0_transfer)

        outputs = {}
        for idx, freq in enumerate(self._frequencies):
            if idx < len(self._frequencies) - 1:
                # get predictions and state up to the time step of information transfer
                slice_timestep = self._slice_timestep[freq]
                lstm_output_slice1, (h_n_slice1, c_n_slice1) = self.lstms[freq](x_d[freq][:-slice_timestep],
                                                                                (h_0_transfer, c_0_transfer))

                # project the states through a hidden layer to the dimensions of the next LSTM
                if self._transfer_mtslstm_states["h"] is not None:
                    h_0_transfer = self.transfer_fcs[f"h_{freq}"](h_n_slice1)
                if self._transfer_mtslstm_states["c"] is not None:
                    c_0_transfer = self.transfer_fcs[f"c_{freq}"](c_n_slice1)

                # get predictions of remaining part and concat results
                lstm_output_slice2, _ = self.lstms[freq](x_d[freq][-slice_timestep:], (h_n_slice1, c_n_slice1))
                lstm_output = torch.cat([lstm_output_slice1, lstm_output_slice2], dim=0)

            else:
                # for highest frequency, we can pass the entire sequence at once
                lstm_output, _ = self.lstms[freq](x_d[freq], (h_0_transfer, c_0_transfer))

            head_out = self.heads[freq](self.dropout(lstm_output.transpose(0, 1)))
            outputs.update({f'{key}_{freq}': value for key, value in head_out.items()})

        return outputs

## LOSS function class

In [70]:
from typing import Dict, List, Tuple

import numpy as np
import torch

from neuralhydrology.training.regularization import BaseRegularization
from neuralhydrology.utils.config import Config

ONE_OVER_2PI_SQUARED = 1.0 / np.sqrt(2.0 * np.pi)


class BaseLoss_v1(torch.nn.Module):
    """Base loss class.

    All losses extend this class by implementing `_get_loss`.

    Parameters
    ----------
    cfg : Config
        The run configuration.
    prediction_keys : List[str]
        List of keys that will be predicted. During the forward pass, the passed `prediction` dict
        must contain these keys. Note that the keys listed here should be without frequency identifier.
    ground_truth_keys : List[str]
        List of ground truth keys that will be needed to compute the loss. During the forward pass, the
        passed `data` dict must contain these keys. Note that the keys listed here should be without
        frequency identifier.
    additional_data : List[str], optional
        Additional list of keys that will be taken from `data` in the forward pass to compute the loss.
        For instance, this parameter can be used to pass the variances that are needed to compute an NSE.
    output_size_per_target : int, optional
        Number of model outputs (per element in `prediction_keys`) connected to a single target variable, by default 1. 
        For example for regression, one output (last dimension in `y_hat`) maps to one target variable. For mixture 
        models (e.g. GMM and CMAL) the number of outputs per target corresponds to the number of distributions 
        (`n_distributions`).
    """

    def __init__(self,
                 prediction_keys: List[str],
                 ground_truth_keys: List[str],
                 additional_data: List[str] = None,
                 output_size_per_target: int = 1):
        super(BaseLoss_v1, self).__init__()
        self._predict_last_n = PREDICT_LAST_N
        self._frequencies = USE_FREQUENCIES
        self._output_size_per_target = output_size_per_target

        self._regularization_terms = []

        # names of ground truth and prediction keys to be unpacked and subset to predict_last_n items.
        self._prediction_keys = prediction_keys
        self._ground_truth_keys = ground_truth_keys

        # subclasses can use this list to register inputs to be unpacked during the forward call
        # and passed as kwargs to _get_loss() without subsetting.
        self._additional_data = []
        if additional_data is not None:
            self._additional_data = additional_data

        # all classes allow per-target weights for multi-target settings. By default, all targets are weighted equally
        if TARGET_WEIGHTS is None:
            weights = torch.tensor([1 / OUTPUT_DIM for _ in range(OUTPUT_DIM)])
        else:
            if len(TARGET_WEIGHTS) == OUTPUT_DIM:
                weights = torch.tensor(TARGET_WEIGHTS)
            else:
                raise ValueError("Number of weights must be equal to the number of target variables")
        self._target_weights = weights

    def forward(self, prediction: Dict[str, torch.Tensor], data: Dict[str, torch.Tensor]) -> torch.Tensor:
        """Calculate the loss.

        Parameters
        ----------
        prediction : Dict[str, torch.Tensor]
            Dictionary of predictions for each frequency. If more than one frequency is predicted,
            the keys must have suffixes ``_{frequency}``. For the required keys, refer to the documentation
            of the concrete loss.
        data : Dict[str, torch.Tensor]
            Dictionary of ground truth data for each frequency. If more than one frequency is predicted,
            the keys must have suffixes ``_{frequency}``. For the required keys, refer to the documentation
            of the concrete loss.

        Returns
        -------
        torch.Tensor
            The calculated loss.
        """
        # unpack loss-specific additional arguments
        kwargs = {key: data[key] for key in self._additional_data}

        losses = []
        prediction_sub, ground_truth_sub = {}, {}
        for freq in self._frequencies:
            if self._predict_last_n[freq] == 0:
                continue  # no predictions for this frequency
            freq_suffix = '' if freq == '' else f'_{freq}'

            # apply predict_last_n and mask for all outputs of this frequency at once
            freq_pred, freq_gt = self._subset_in_time(
                {key: prediction[f'{key}{freq_suffix}'] for key in self._prediction_keys},
                {key: data[f'{key}{freq_suffix}'] for key in self._ground_truth_keys}, self._predict_last_n[freq])

            # remember subsets for multi-frequency component
            prediction_sub.update({f'{key}{freq_suffix}': freq_pred[key] for key in freq_pred.keys()})
            ground_truth_sub.update({f'{key}{freq_suffix}': freq_gt[key] for key in freq_gt.keys()})

            for n_target, weight in enumerate(self._target_weights):
                # subset the model outputs and ground truth corresponding to this particular target
                target_pred, target_gt = self._subset_target(freq_pred, freq_gt, n_target)

                # model hook to subset additional data, which might be different for different losses
                kwargs_sub = self._subset_additional_data(kwargs, n_target)

                loss = self._get_loss(target_pred, target_gt, **kwargs_sub)
                losses.append(loss * weight)

        loss = torch.sum(torch.stack(losses))
        for regularization in self._regularization_terms:
            loss = loss + regularization(prediction_sub, ground_truth_sub,
                                         {k: v for k, v in prediction.items() if k not in self._prediction_keys})
        return loss

    @staticmethod
    def _subset_in_time(prediction: Dict[str, torch.Tensor], ground_truth: Dict[str, torch.Tensor],
                        predict_last_n: int) -> Tuple[Dict[str, torch.Tensor], Dict[str, torch.Tensor]]:
        ground_truth_sub = {key: gt[:, -predict_last_n:, :] for key, gt in ground_truth.items()}
        prediction_sub = {key: pred[:, -predict_last_n:, :] for key, pred in prediction.items()}

        return prediction_sub, ground_truth_sub

    def _subset_target(self, prediction: Dict[str, torch.Tensor], ground_truth: Dict[str, torch.Tensor],
                       n_target: int) -> Tuple[Dict[str, torch.Tensor], Dict[str, torch.Tensor]]:
        # determine which output neurons correspond to the n_target target variable
        start = n_target * self._output_size_per_target
        end = (n_target + 1) * self._output_size_per_target
        prediction_sub = {key: pred[:, :, start:end] for key, pred in prediction.items()}

        # subset target by slicing to keep shape [bs, seq, 1]
        ground_truth_sub = {key: gt[:, :, n_target:n_target + 1] for key, gt in ground_truth.items()}

        return prediction_sub, ground_truth_sub

    @staticmethod
    def _subset_additional_data(additional_data: Dict[str, torch.Tensor], n_target: int) -> Dict[str, torch.Tensor]:
        # by default, nothing happens
        return additional_data

    def _get_loss(self, prediction: Dict[str, torch.Tensor], ground_truth: Dict[str, torch.Tensor], **kwargs):
        raise NotImplementedError

    def set_regularization_terms(self, regularization_modules: List[BaseRegularization]):
        """Register the passed regularization terms to be added to the loss function.

        Parameters
        ----------
        regularization_modules : List[BaseRegularization]
            List of regularization functions to be added to the loss during `forward`.
        """
        self._regularization_terms = regularization_modules


class MaskedMSELoss_v1(BaseLoss_v1):
    """Mean squared error loss.

    To use this loss in a forward pass, the passed `prediction` dict must contain
    the key ``y_hat``, and the `data` dict must contain ``y``.

    Parameters
    ----------
    cfg : Config
        The run configuration.
    """

    def __init__(self):
        super(MaskedMSELoss_v1, self).__init__(prediction_keys=['y_hat'], ground_truth_keys=['y'])

    def _get_loss(self, prediction: Dict[str, torch.Tensor], ground_truth: Dict[str, torch.Tensor], **kwargs):
        mask = ~torch.isnan(ground_truth['y'])
        loss = 0.5 * torch.mean((prediction['y_hat'][mask] - ground_truth['y'][mask])**2)
        return loss


class MaskedRMSELoss_v1(BaseLoss_v1):
    """Root mean squared error loss.

    To use this loss in a forward pass, the passed `prediction` dict must contain
    the key ``y_hat``, and the `data` dict must contain ``y``.

    Parameters
    ----------
    cfg : Config
        The run configuration.
    """

    def __init__(self):
        super(MaskedRMSELoss_v1, self).__init__(prediction_keys=['y_hat'], ground_truth_keys=['y'])

    def _get_loss(self, prediction: Dict[str, torch.Tensor], ground_truth: Dict[str, torch.Tensor], **kwargs):
        mask = ~torch.isnan(ground_truth['y'])
        loss = torch.sqrt(0.5 * torch.mean((prediction['y_hat'][mask] - ground_truth['y'][mask])**2))
        return loss


In [80]:
def _get_predictions_and_loss(model, data, l_obj):
    predictions = model(data)
    loss = l_obj(predictions, data)
    return predictions, loss.item()

def _subset_targets(data, predictions, predict_last_n, freq):
    y_hat_sub = predictions[f'y_hat{freq}'][:, -predict_last_n:, :]
    y_sub = data[f'y{freq}'][:, -predict_last_n:, :]
    return y_hat_sub, y_sub


def _evaluate(model, loader, l_obj, frequencies):
    """Evaluate model"""
    predict_last_n = PREDICT_LAST_N
    # if isinstance(predict_last_n, int):
    #     predict_last_n = {frequencies[0]: predict_last_n}  # if predict_last_n is int, there's only one frequency

    preds, obs = {}, {}
    losses = []
    with torch.no_grad():
        for data in loader:

            # for key in data:
            #     data[key] = data[key].to(self.device)
            predictions, loss = _get_predictions_and_loss(model, data, l_obj)

            for freq in frequencies:
                freq_key = f'_{freq}'
                y_hat_sub, y_sub = _subset_targets(data, predictions, predict_last_n[freq], freq_key)

                if freq not in preds:
                    preds[freq] = y_hat_sub.detach().cpu()
                    obs[freq] = y_sub.cpu()
                else:
                    preds[freq] = torch.cat((preds[freq], y_hat_sub.detach().cpu()), 0)
                    obs[freq] = torch.cat((obs[freq], y_sub.detach().cpu()), 0)

            losses.append(loss)

        for freq in preds.keys():
            preds[freq] = preds[freq].numpy()
            obs[freq] = obs[freq].numpy()

    # set to NaN explicitly if all losses are NaN to avoid RuntimeWarning
    mean_loss = np.nanmean(losses) if len(losses) > 0 and not all(np.isnan(l) for l in losses) else np.nan
    return preds, obs, mean_loss

In [81]:
new_model = AiBEDO_MTSLSTM()

In [82]:
new_model._input_sizes

{'1D': 5, '1H': 16}

In [83]:
new_model

AiBEDO_MTSLSTM(
  (lstms): ModuleDict(
    (1D): LSTM(5, 20)
    (1H): LSTM(16, 20)
  )
  (transfer_fcs): ModuleDict(
    (c_1D): Linear(in_features=20, out_features=20, bias=True)
    (h_1D): Linear(in_features=20, out_features=20, bias=True)
  )
  (heads): ModuleDict(
    (1D): Regression(
      (net): Sequential(
        (0): Linear(in_features=20, out_features=1, bias=True)
      )
    )
    (1H): Regression(
      (net): Sequential(
        (0): Linear(in_features=20, out_features=1, bias=True)
      )
    )
  )
  (dropout): Dropout(p=0.4, inplace=False)
)

In [84]:
optimizer = torch.optim.Adam(new_model.parameters(), lr=1e-2)

In [85]:
loss_obj = MaskedMSELoss_v1()

In [86]:
for epoch in range(50):
    new_model.train()
    pbar = tqdm(dl_train)
    
    for data in pbar:
        predictions = new_model(data)
        loss_val = loss_obj(predictions, data)
        optimizer.zero_grad()
        loss_val.backward()
        
        torch.nn.utils.clip_grad_norm_(new_model.parameters(), 1)
        optimizer.step()
        
        pbar.set_postfix_str(f"Loss: {loss_val.item():.4f}")
    
    if epoch%5 == 0:    
        ## validate
        new_model.eval()
        _, _, avg_val_loss = _evaluate(new_model, dl_val, loss_obj, USE_FREQUENCIES)
        print("Avg. validation loss", avg_val_loss)

100%|██████████| 11/11 [00:03<00:00,  2.82it/s, Loss: 0.3788]


Avg. validation loss 0.35441458225250244


100%|██████████| 11/11 [00:03<00:00,  2.81it/s, Loss: 0.3289]


Avg. validation loss 0.24830869138240813


100%|██████████| 11/11 [00:04<00:00,  2.67it/s, Loss: 0.2100]


...

100%|██████████| 11/11 [00:03<00:00,  2.79it/s, Loss: 0.1002]


Avg. validation loss 0.21514617800712585


100%|██████████| 11/11 [00:03<00:00,  2.79it/s, Loss: 0.1109]


Avg. validation loss 0.23508144915103912


100%|██████████| 11/11 [00:03<00:00,  2.79it/s, Loss: 0.0715]


In [104]:
from ruamel.yaml import YAML


In [111]:
yml_path = Path('aibedo.yml')

In [112]:
if yml_path.exists():
    with yml_path.open('r') as fp:
        yaml = YAML(typ="safe")
        cfg_test = yaml.load(fp)
else:
    raise FileNotFoundError(yml_path)


    
    

In [117]:
for k in cfg_test.keys():
    print(k, cfg_test[k])

lowres_input_dimension 5
highres_input_dimension 16
output_dim 1
shared_mtslstm False
transfer_mtslstm_states {'h': 'linear', 'c': 'linear'}
output_activation linear
hidden_size 20
initial_forget_bias 3
output_dropout 0.4
device cpu
optimizer Adam
loss MSE
regularization ['tie_frequencies']
batch_size 256
epochs 50
clip_gradient_norm 1
predict_last_n {'1D': 1, '1H': 24}
seq_length {'1D': 365, '1H': 336}
num_workers 8
log_interval 5
log_tensorboard False
log_n_figures 0
save_weights_every 1


In [ ]:
LR_INPUT_DIM = 5
HR_INPUT_DIM = 16
INPUT_SIZE = {'1D': 5, '1H': 16}
OUTPUT_DIM = 1
HIDDEN_SIZE = {'1D': 20, '1H': 20}
INITIAL_FORGET_BIAS = 3
SHARED_MTSLSTM = False
OUTPUT_DROPOUT = 0.4

TRANSFER_MTSLSTM_STATES = {'h': 'linear', 'c': 'linear'}

USE_FREQUENCIES = ['1D', '1H']
OUTPUT_ACTIVATION = 'linear'

DEVICE = 'cpu'

SEQ_LENGTH = {'1D': 365, '1H': 336}
PREDICT_LAST_N = {'1D': 1, '1H': 24}

SLICE_TIMESTEP = {'1D': 14}
FREUENCY_FACTORS = [24]

#weights per target
TARGET_WEIGHTS = [1.0]

In [120]:
class AiBEDOConfig():
    def __init__(self, yml_path: Path):
        
        if yml_path.exists():
            with yml_path.open('r') as fp:
                yaml = YAML(typ="safe")
                _cfg_dict = yaml.load(fp)
        else:
            raise FileNotFoundError(yml_path)
        
        self.num_variables = len(_cfg_dict)
        self.input_size = _cfg_dict['input_size']
        self.output_dim = _cfg_dict['output_dim']
        self.hidden_size = _cfg_dict['hidden_size']
        
        self.initial_forget_bias = _cfg_dict['initial_forget_bias']
        self.shared_mtslstm = _cfg_dict['shared_mtslstm']
        self.output_dropout = _cfg_dict['output_dropout']
        
        self.transfer_mtslstm_states = _cfg_dict['transfer_mtslstm_states']
        self.use_frequencies = _cfg_dict['use_frequencies']
        self.output_activation = _cfg_dict['output_activation']
        
        self.device = _cfg_dict['device']
        
        self.seq_length = _cfg_dict['seq_length']
        self.predict_last_n = _cfg_dict['predict_last_n']
        self.slice_timestep = _cfg_dict['slice_timestep']
        self.frequency_factors = _cfg_dict['frequency_factors']
        
        self.target_weights = _cfg_dict['target_weights']

        
         
        

In [121]:
conf_v1 = AiBEDOConfig(Path('aibedo.yml'))

In [125]:
conf_v1.frequency_factors

[24]

In [119]:
len(cfg_test)

23

In [94]:
test_conf = Config(Path('example.yml'), dev_mode=True)

In [97]:
test_conf.a_dictionary

AttributeError: 'Config' object has no attribute 'a_dictionary'